In [2]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

source("code/enrichment_fxns.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




Here I perform enrichment analysis to find modules enriched for cell type markers. 

These modules will later be used to correlate with exon PSI to define cell type-specific exons.

In [3]:
network_dir <- "tasic_2018_ALM_STAR_25pcntCells_100SD_200samples_log2_pseudobulk_Modules"
network_dataset <- "25pcntCells_100SD_200samples_log2_pseudobulk"

In [4]:
mod_def <- "PosBC"
unique <- FALSE
lfc_threshold <- 5

### Prep DE genes

#### Pairwise tests

In [5]:
pairwise_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")
pairwise_res_list <- pairwise_res_list[!grepl("L4|Peri|CR|Meis",  names(pairwise_res_list))]
pairwise_ctype_genes <- prep_DE_genes(pairwise_res_list, lfc_threshold, pairwise=TRUE, unique=unique)

#### 1 vs. pooled tests

In [6]:
pooled_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")
pooled_ctype_genes <- prep_DE_genes(pooled_res_list, lfc_threshold, pairwise=FALSE, unique=unique)

#### 1 vs. mean tests

In [7]:
mean_res_list <- readRDS("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream.RDS")
mean_ctype_genes <- prep_DE_genes(mean_res_list, lfc_threshold, pairwise=FALSE, unique=unique)

### Enrichment results using pairwise DE genes

In [8]:
pairwise_enrichments_df <- get_module_enrichments(network_dir, pairwise_ctype_genes, mod_def)

In [10]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pairwise_top_mods_df <- pairwise_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)
    # group_by(Cell_type) %>%
    # slice_min(Qval, with_ties=FALSE) %>%
    # group_by(Network, Module) %>%
    # arrange(Network) %>%
    # slice_min(Qval, with_ties=FALSE) %>%
    # arrange(Qval)

In [11]:
pairwise_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Astro,0.000000e+00,0.000000e+00,brown,Bicor-None_signum0.488_minSize10_merge_ME_0.9_20151
Endo,0.000000e+00,0.000000e+00,yellow,Bicor-None_signum0.488_minSize10_merge_ME_0.9_20151
Macrophage,0.000000e+00,0.000000e+00,turquoise,Bicor-None_signum0.488_minSize10_merge_ME_0.9_20151
Oligo,0.000000e+00,0.000000e+00,blue,Bicor-None_signum0.488_minSize10_merge_ME_0.9_20151
SMC,1.890679e-309,8.307985e-307,pink,Bicor-None_signum0.488_minSize8_merge_ME_0.9_20151
VLMC,1.137089e-290,4.697633e-288,green,Bicor-None_signum0.585_minSize6_merge_ME_0.9_20151
L5_IT,7.845416e-182,2.478536e-179,royalblue,Bicor-None_signum0.668_minSize10_merge_ME_0.9_20151
Pvalb,2.609246e-157,7.982311e-155,darkturquoise,Bicor-None_signum0.488_minSize10_merge_ME_0.9_20151
Lamp5,1.899123e-128,5.071602e-126,yellowgreen,Bicor-None_signum0.874_minSize5_merge_ME_0.9_20151


In [12]:
if (unique) {
    write.csv(pairwise_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(pairwise_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Enrichment results using 1 vs. pooled DE genes

In [29]:
pooled_enrichments_df <- get_module_enrichments(network_dir, pooled_ctype_genes, mod_def)

In [30]:
# pooled_enrichments_df %>%
#     group_by(Cell_type) %>%
#     slice_min(Qval) %>%
#     group_by(Cell_type) %>%
#     arrange(Network, Module) %>%
#     arrange(Qval)

In [ ]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

pooled_top_mods_df <- pooled_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)

In [32]:
pooled_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,1.535211e-162,1.564933e-157,turquoise,Bicor-None_signum0.52_minSize5_merge_ME_0.9_20151
Endo,1.533268e-153,3.125904e-149,brown,Bicor-None_signum0.34_minSize6_merge_ME_0.9_20151
Peri,2.806615e-151,4.087073e-147,saddlebrown,Bicor-None_signum0.791_minSize6_merge_ME_0.9_20151
Astro,2.881896e-142,1.958460e-138,brown,Bicor-None_signum0.43_minSize6_merge_ME_0.9_20151
Oligo,2.173744e-129,6.330937e-126,green,Bicor-None_signum0.34_minSize6_merge_ME_0.9_20151
VLMC,3.474881e-61,2.643399e-58,green,Bicor-None_signum0.931_minSize10_merge_ME_0.9_20151
NP,1.502325e-60,1.126037e-57,darkturquoise,Bicor-None_signum0.791_minSize6_merge_ME_0.9_20151
Sst,1.733735e-58,1.262357e-55,lightcoral,Bicor-None_signum0.52_minSize10_merge_ME_0.9_20151
L6_CT,5.651569e-47,3.310910e-44,darkorange2,Bicor-None_signum0.52_minSize10_merge_ME_0.9_20151


In [33]:
if (unique) {
    write.csv(pooled_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(pooled_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Enrichment results using 1 vs. mean DE genes

In [34]:
mean_enrichments_df <- get_module_enrichments(network_dir, mean_ctype_genes, mod_def)

In [35]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

mean_top_mods_df <- mean_enrichments_df %>%
    group_by(Network, Module) %>%
    slice_min(Qval) %>%
    group_by(Cell_type) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)

In [36]:
mean_top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
Macrophage,8.446361e-214,4.304941e-209,turquoise,Bicor-None_signum0.52_minSize5_merge_ME_0.9_20151
Astro,1.703460e-205,3.472878e-201,brown,Bicor-None_signum0.43_minSize6_merge_ME_0.9_20151
Oligo,1.900602e-188,7.451530e-185,yellow,Bicor-None_signum0.931_minSize5_merge_ME_0.9_20151
Endo,4.590132e-178,9.955312e-175,brown,Bicor-None_signum0.34_minSize6_merge_ME_0.9_20151
Peri,1.277614e-171,2.325622e-168,salmon,Bicor-None_signum0.791_minSize10_merge_ME_0.9_20151
NP,5.941706e-101,4.964539e-98,salmon,Bicor-None_signum0.931_minSize4_merge_ME_0.9_20151
VLMC,1.795339e-100,1.487884e-97,green,Bicor-None_signum0.931_minSize10_merge_ME_0.9_20151
L6_IT,3.473856e-85,2.662489e-82,grey60,Bicor-None_signum0.931_minSize3_merge_ME_0.9_20151
Pvalb,2.454496e-77,1.702051e-74,ivory,Bicor-None_signum0.791_minSize5_merge_ME_0.9_20151


In [37]:
if (unique) {
    write.csv(mean_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(mean_top_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}

### Compare enrichment results

In [38]:
pooled_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pooled_top_mods_df$Network))
pairwise_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", pairwise_top_mods_df$Network))
mean_top_mods_df$Network_short <- gsub("Bicor-None_", "", gsub("_merge_ME_0.9_20151", "", mean_top_mods_df$Network))

cols <- c("Cell_type", "Qval", "Module", "Network_short")

all_top_mods_df <- merge(
    merge(
        pooled_top_mods_df[,cols], 
        pairwise_top_mods_df[,cols], 
        by="Cell_type", all=TRUE
    ), 
    mean_top_mods_df[,cols], by="Cell_type", all=TRUE
)

In [39]:
all_top_mods_df <- all_top_mods_df[,order(colnames(all_top_mods_df))]
all_top_mods_df

Cell_type,Module,Module.x,Module.y,Network_short,Network_short.x,Network_short.y,Qval,Qval.x,Qval.y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Astro,brown,brown,yellow,signum0.43_minSize6,signum0.43_minSize6,signum0.34_minSize10,3.472878e-201,1.958460e-138,0.000000e+00
CR,salmon,tan,salmon,signum0.52_minSize5,signum0.34_minSize6,signum0.52_minSize5,2.111882e-24,7.359438e-12,4.451718e-49
Endo,brown,brown,brown,signum0.34_minSize6,signum0.34_minSize6,signum0.34_minSize10,9.955312e-175,3.125904e-149,0.000000e+00
L2_3_IT,grey60,lavenderblush2,lavenderblush2,signum0.791_minSize5,signum0.791_minSize4,signum0.791_minSize4,9.689588e-55,6.269337e-20,1.255658e-56
L4,tan,cornsilk3,pink,signum0.791_minSize10,signum0.378_minSize6,signum0.378_minSize8,1.471659e-25,4.347131e-09,2.643666e-08
L5_IT,lightcyan,ivory,plum4,signum0.791_minSize6,signum0.791_minSize4,signum0.378_minSize10,2.308468e-72,1.857841e-27,1.538018e-64
L5_PT,royalblue,royalblue,royalblue,signum0.52_minSize10,signum0.52_minSize10,signum0.52_minSize10,1.218179e-27,3.086790e-21,5.823250e-29
L6_CT,darkslateblue,darkorange2,navajowhite1,signum0.791_minSize5,signum0.52_minSize10,signum0.791_minSize3,1.343667e-51,3.310910e-44,4.425526e-44
L6_IT,grey60,lightpink4,lightpink4,signum0.931_minSize3,signum0.52_minSize10,signum0.52_minSize10,2.662489e-82,1.073439e-25,9.777395e-32


### Select the module from the lowest Qval per cell type

In [40]:
cols <- c("Cell_type", "Qval", "Module", "Network_short")

In [41]:
# Select the module from the lowest Qval per cell type

pooled_top_mods_df$DE_Test <- "Pooled"
pairwise_top_mods_df$DE_Test <- "Pairwise"
mean_top_mods_df$DE_Test <- "Mean"

all_top_mods_df <- rbind(pooled_top_mods_df, pairwise_top_mods_df, mean_top_mods_df)

In [42]:
top_qval_mods_df <- all_top_mods_df %>%
    group_by(Cell_type) %>%
    slice_min(Qval, with_ties=FALSE)

top_qval_mods_df[, c(cols, "DE_Test")] %>% arrange(Qval)

Cell_type,Qval,Module,Network_short,DE_Test
<chr>,<dbl>,<chr>,<chr>,<chr>
Astro,0.000000e+00,yellow,signum0.34_minSize10,Pairwise
Endo,0.000000e+00,brown,signum0.34_minSize10,Pairwise
Macrophage,0.000000e+00,turquoise,signum0.34_minSize10,Pairwise
Oligo,1.320634e-287,yellow,signum0.931_minSize5,Pairwise
VLMC,2.446324e-203,red,signum0.975_minSize3,Pairwise
Peri,2.325622e-168,salmon,signum0.791_minSize10,Mean
SMC,8.864733e-148,green,signum0.975_minSize4,Pairwise
Pvalb,2.215634e-120,ivory,signum0.791_minSize5,Pairwise
NP,4.964539e-98,salmon,signum0.931_minSize4,Mean


In [43]:
top_qval_mods_df

Network,kME_path,ME_path,Pval,Cell_type,Module,Qval,Network_short,DE_Test
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
Bicor-None_signum0.34_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.34_minSize10_merge_ME_0.9_20151/kME_table_09-45-53.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.34_minSize10_merge_ME_0.9_20151/Module_eigengenes_09-45-53.csv,0.000000e+00,Astro,yellow,0.000000e+00,signum0.34_minSize10,Pairwise
Bicor-None_signum0.52_minSize5_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.52_minSize5_merge_ME_0.9_20151/kME_table_05-03-38.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.52_minSize5_merge_ME_0.9_20151/Module_eigengenes_05-03-38.csv,1.091792e-51,CR,salmon,4.451718e-49,signum0.52_minSize5,Pairwise
Bicor-None_signum0.34_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.34_minSize10_merge_ME_0.9_20151/kME_table_09-45-53.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.34_minSize10_merge_ME_0.9_20151/Module_eigengenes_09-45-53.csv,0.000000e+00,Endo,brown,0.000000e+00,signum0.34_minSize10,Pairwise
Bicor-None_signum0.791_minSize4_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.791_minSize4_merge_ME_0.9_20151/kME_table_04-53-11.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.791_minSize4_merge_ME_0.9_20151/Module_eigengenes_04-53-11.csv,2.944027e-59,L2_3_IT,lavenderblush2,1.255658e-56,signum0.791_minSize4,Pairwise
Bicor-None_signum0.791_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.791_minSize10_merge_ME_0.9_20151/kME_table_05-02-40.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.791_minSize10_merge_ME_0.9_20151/Module_eigengenes_05-02-40.csv,6.034701e-28,L4,tan,1.471659e-25,signum0.791_minSize10,Mean
Bicor-None_signum0.791_minSize6_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.791_minSize6_merge_ME_0.9_20151/kME_table_04-59-50.csv,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.791_minSize6_merge_ME_0.9_20151/Module_eigengenes_04-59-50.csv,3.396938e-75,L5_IT,lightcyan,2.308468e-72,signum0.791_minSize6,Mean
Bicor-None_signum0.52_minSize10_merge_ME_0.9_20151,/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/tasic_2018_ALM_STAR_20pcntCells_35SD_200samples_log2_pseudobulk_Modules/Bicor-None_signum0.52_minSize10_merge_ME_0.9_20151/kME_table_05-52-51.csv,/mnt/lareaulab/reliscu/project

In [44]:
if (unique) {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_unique_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
} else {
    write.csv(top_qval_mods_df, file=paste0("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_DE_genes_dream_", network_dataset, "_", mod_def, "_top_Qval_modules.csv"), row.names=FALSE, quote=FALSE)
}
